In [273]:
import pandas as pd 
from pathlib import Path
import re


In [274]:
folder =  Path("../results/synthetic/1000/set4/")
folder.exists()


True

In [275]:
def get_id(f):
    return re.sub("_x[0-9]*", "_x[0-9]*", str(f).split("/")[-1]) 


MODELS = list(set([get_id(f) for f in list(folder.rglob("*.csv"))]))
len(MODELS)

98

In [276]:

def fix_file(file):
    with open(file, "r") as f:
        data = f.read()

    if '", ' in data or 'treewdith' in data: 
        with open(file, "w") as f:
            f.write(re.sub('", ', '|', re.sub('treewdith', 'treewidth', data)))
            
            
    

for f in folder.rglob("*.csv"):
    fix_file(f)

In [270]:
def aggregate_data(m):
    files = list(folder.rglob(m))

    # the first should be the non additional
    files = [f for f in files if "additional" not in str(f)] + [f for f in files if "additional" in str(f)]

    data = [pd.read_csv(f) for f in files]
    full_data = data[0]

    for i in range(1,len(files)):
        d = data[i]
        pns_count = len([c for c in full_data.columns if re.match("pns_[0-9]+", c)])

        full_data["time_learn_runs"] = pns_count

        new_names_pns = {c:f'pns_{int(c.split("pns_")[-1]) + pns_count}' for c in d.columns if re.match("pns_[0-9]+", c)}
        d = d.rename(columns=new_names_pns)

        new_names_trs = {c:f'trajectory_size_{int(c.split("trajectory_size_")[-1]) + pns_count}' for c in d.columns if re.match("trajectory_size_[0-9]+", c)}
        d = d.rename(columns=new_names_trs)

        d_cols = ["method", "selector", "ps1", "treewidth", "bias_endo_treewidth", "endo_treewidth", "bias_treewidth", "time_learn"] + list(new_names_pns.values()) + list(new_names_trs.values())
        full_data = pd.concat([full_data, d[d_cols]], axis=1 )


    df = full_data
    pns_u = df[[c for c in df.columns if re.match("pns_[0-9]*", c)]].max(axis=1)
    pns_l = df[[c for c in df.columns if re.match("pns_[0-9]*", c)]].min(axis=1)

    df.pns_u = pns_u
    df.pns_l = pns_l

    full_data = df

    num_iter = len([c for c in full_data.columns if re.match("pns_[0-9]+", c)])
    num_iter

    new_name = m.replace("_x[0-9]*", f"_x{num_iter}")
    new_folder = Path(str(folder)+"_aggregated")
    if not new_folder.exists():
        new_folder.mkdir()

    new_file = Path(new_folder, new_name)
    print(new_file)
    full_data.to_csv(new_file)

In [271]:
for m in MODELS:
    aggregate_data(m)

../results/synthetic/1000/set4_aggregated/rand13_mk0_nEndo5_k05_3_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo5_k05_2_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo4_k075_3_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo6_k10_4_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo4_k075_4_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo5_k05_5_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk0_nEndo4_k05_1_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk1_maxDist2_nEndo6_k075_4_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_aggregated/rand13_mk0_nEndo4_k05_5_mIter300_wtrue_sparents3_x80_0.csv
../results/synthetic/1000/set4_agg